___
<a href='http://www.pucminas.br/Pos-Graduacao/Paginas/default.aspx'> <img src='assets/puc-logo.png' /></a>
___

## Disciplina : Introdução às Linguagens Estatísticas     
** Professor: Raphael Rodrigues Campos **

** Aluno: Antonio Carneiro de Albuquerque Netto **

** Matrícula: 83026 **
<center><h1>Trabalho Prático</h1></center>

### Imports e header da aplicação

In [67]:
import json
import string
import pandas as pd
import numpy as np
from pprint import pprint

pandasDF = pd.read_json('perfis.json')

pandasDF.head()

,education,experiences,gender,languages,skills
0,"[{u'date': u'2016 – 2016', u'school': u'Stanfo...","[{u'date': u'set de 2011 – o momento', u'posit...",MALE,"[{u'proficiency': u'Nível avançado', u'name': ...","[{u'endorsement_count': u'+ de 99', u'name': u..."
1,"[{u'date': u'2015 – 2015', u'school': u'Centro...","[{u'date': u'jul de 2017 – o momento', u'posit...",FEMALE,"[{u'name': u'Idioma Inglês'}, {u'name': u'Idio...","[{u'endorsement_count': u'45', u'name': u'Plan..."
2,"[{u'date': u'2016 – 2019', u'school': u'Univer...","[{u'date': u'jul de 2017 – o momento', u'posit...",MALE,"[{u'proficiency': u'Nível avançado', u'name': ...","[{u'endorsement_count': u'8', u'name': u'JavaS..."
3,"[{u'date': u'2014 – 2019', u'school': u'Centro...","[{u'date': u'ago de 2017 – o momento', u'posit...",FEMALE,[{u'proficiency': u'Nível básico a intermediár...,"[{u'endorsement_count': u'2', u'name': u'Net P..."
4,"[{u'date': u'2011 – 2016', u'school': u'Univer...","[{u'date': u'mar de 2017 – o momento', u'posit...",MALE,"[{u'proficiency': u'Fluente ou nativo', u'name...","[{u'endorsement_count': u'13', u'name': u'Micr..."


### Leitura e criação dos DataFrames

**Função Auxiliar para montagem das tabelas e tratamento de valores desnecessários em algumas colunas**

In [61]:
## Func Aux
def setupTable(myColumn):
    
    is_list = myColumn.apply(lambda x: isinstance(x, list))
    only_lists = myColumn[is_list]
    
    ## Tratamento na Languages
    if myColumn.name == 'languages':
        for personLangs in only_lists:
            for lang in personLangs:
                lang['name'] = lang['name'].replace('Idioma\n','')
                if 'proficiency' in lang:
                    proficiency = lang['proficiency'].encode('utf-8')
                    lang['proficiency'] = proficiency.replace('N\xc3\xadvel ','')
    
    ## Tratamento na Education
    if myColumn.name == 'education':
        for personEducation in only_lists:
            for eduItem in personEducation:
                ## Removendo parte desnecessária da string degree e major
                if 'degree' in eduItem:
                    degreeName = eduItem['degree'].replace('Nome do diploma\n','')
                    eduItem['degree'] = degreeName
                if 'major' in eduItem:
                    majorName = eduItem['major'].encode('utf-8').replace('Área de estudo\n','')
                    eduItem['major'] = majorName
    
    #return only_lists[0]
    df = pd.DataFrame.from_dict(only_lists.sum())

    df['personId'] = np.repeat(only_lists.index,
                                   only_lists.apply(lambda x: len(x)))
    return df

### Tabela de Languages

In [62]:
## LANG table
setupTable(pandasDF.languages)

,name,proficiency,personId
0,Espanhol,avançado,0
1,Inglês,Fluente ou nativo,0
2,Português,Fluente ou nativo,0
3,Inglês,NaN,1
4,Italiano,NaN,1
5,English,avançado,2
6,Portuguese,Fluente ou nativo,2
7,Espanhol,básico a intermediário,3
8,Inglês,avançado,3
9,Português,Fluente ou nativo,3


### Tabela de Experiences

In [63]:
## EXP table
setupTable(pandasDF.experiences)

,company,date,location,position,personId
0,Trojan Technologies,set de 2011 – o momento,"Belo Horizonte e Região, Brasil",CEO and Founder,0
1,NaN,NaN,NaN,NaN,0
2,NaN,NaN,NaN,NaN,0
3,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,0
5,NaN,NaN,NaN,NaN,0
6,Any Company,mar de 2016 – o momento,NaN,Endeavor Entrepreneur,0
7,Any Company,out de 2015 – o momento,"Belo Horizonte e Região, Brasil",Investor and Board Member,0
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0


### Tabela de Skills

In [64]:
## Skills table
setupTable(pandasDF.skills)

,endorsement_count,name,personId
0,+ de 99,Empresas startups,0
1,NaN,NaN,0
2,NaN,NaN,0
3,+ de 99,Empreendedorismo,0
4,NaN,NaN,0
5,NaN,NaN,0
6,97,Start-ups,0
7,NaN,NaN,0
8,NaN,NaN,0
9,73,Entrepreneurship,0


### Tabela de Education

In [65]:
## Education table
setupTable(pandasDF.education)

,date,degree,major,school,personId
0,2016 – 2016,"Executive Education, Innovation & Entrepreneur...",NaN,Stanford University,0
1,2007 – 2010,Economics,Economics,Universidade Federal de Minas Gerais,0
2,2015 – 2015,Master of Business Administration (MBA),Gestão de Negócios,Centro Universitário Newton Paiva,1
3,2009 – 2012,Bacharel em Design de Ambientes,Design de Interiores,Universidade do Estado de Minas Gerais,1
4,2016 – 2019,Bachelor’s Degree,Computer Technology/Computer Systems Technology,Universidade Federal de Minas Gerais,2
5,2013 – 2015,Technician,"Informatics, COLTEC",Universidade Federal de Minas Gerais,2
6,2014 – 2019,Engenharia de Produção Civil,NaN,Centro Federal de Educação Tecnológica de Mina...,3
7,2011 – 2016,Bachelor of Science (B.S.),Chemical Engineering,Universidade Federal de Minas Gerais,4
8,2015 – 2015,Bachelor of Science (B.S.),Chemical Engineering,California State University-Long Beach,4
9,2011 – 2011,Pós Graduação em Gestão Estratégica de Negócios,Economia e Gestão Empresarial,Universidade FUMEC,5
